![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>Model Editing  <a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/mjspec.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

This notebook provides an introductory tutorial for model editing in MuJoCo using the `mjSpec` API. This notebook assumes that the reader is already familiar with MuJoCo basic concepts, as demostrated in the [introductory tutorial](https://github.com/google-deepmind/mujoco?tab=readme-ov-file#getting-started). Documentation for this API can be found in the [Model Editing](https://mujoco.readthedocs.io/en/latest/programming/modeledit.html) chapter in the documentation (C API) and in the [Python chapter](https://mujoco.readthedocs.io/en/latest/python.html#model-editing). Here we use the Python API.

The goal of the API is to allow users to easily interact with and modify MuJoCo
models in Python, similarly to what the JavaScript DOM does for HTML.

<!-- Copyright 2024 DeepMind Technologies Limited

     Licensed under the Apache License, Version 2.0 (the "License");
     you may not use this file except in compliance with the License.
     You may obtain a copy of the License at

         http://www.apache.org/licenses/LICENSE-2.0

     Unless required by applicable law or agreed to in writing, software
     distributed under the License is distributed on an "AS IS" BASIS,
     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
     See the License for the specific language governing permissions and
     limitations under the License.
-->


## All imports

In [ ]:
!pip install mujoco

# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco as mj
  mj.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Other imports and helper functions
import time
import itertools
import numpy as np
import typing
import dataclasses

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()

from pygments import highlight
from pygments.lexers import XmlLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML, display

def print_xml(xml_string):
    formatter = HtmlFormatter(style='lovelace')
    highlighted_xml = highlight(xml_string, XmlLexer(), formatter)
    display(HTML(f"<style>{formatter.get_style_defs()}</style>{highlighted_xml}"))

import matplotlib.pyplot as plt
import matplotlib.animation as animation

def plot_sensor_data(sensor_data,limit):
  # Create a figure and axis
  fig, ax = plt.subplots()
  line, = ax.plot([], [], lw=2)
  ax.set_xlim(0, 50)  # Set x-axis range
  ax.set_ylim(limit[0],limit[1])  # Set y-axis range
  ax.set_title("Sensor Data Animation")
  ax.set_xlabel("Time")
  ax.set_ylabel("Sensor Value")

  # Initialize the plot with empty data
  def init():
      line.set_data([], [])
      return line,

  # Update function for the animation
  def update(frame):
      x_data = np.linspace(0, 50, frame)  # Generate x values (time)
      y_data = sensor_data[:frame]  # Corresponding sensor data
      line.set_data(x_data, y_data)
      return line,

  # Create the animation
  ani = animation.FuncAnimation(fig, update, frames=len(sensor_data), init_func=init, blit=True, interval=50)

  # Display the animation
  plt.show()


def render(model, data=None, height=250):
  if data is None:
    data = mj.MjData(model)
  with mj.Renderer(model, 480, 640) as renderer:
    mj.mj_forward(model, data)
    renderer.update_scene(data)
    media.show_image(renderer.render(), height=height)


# Parsing XML to `mjSpec` and compiling to `mjModel`


Unlike `mj_loadXML` which combines parsing and compiling, when using `mjSpec`, parsing and compiling are separate, allowing for editing steps:

In [ ]:
#@title A static model, from string {vertical-output: true}

static_model = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
spec = mj.MjSpec.from_string(static_model)
model = spec.compile()
render(model)

# Change the mjSpec, re-compile and re-render
geoms = spec.worldbody.find_all(mj.mjtObj.mjOBJ_GEOM)
geoms[0].name = 'blue_box'
geoms[0].rgba = [0, 0, 1, 1]
geoms[1].name = 'yellow_sphere'
geoms[1].rgba = [1, 1, 0, 1]
spec.worldbody.add_geom(name='magenta cylinder',
                        type=mj.mjtGeom.mjGEOM_CYLINDER,
                        rgba=[1, 0, 1, 1],
                        pos=[-.2, 0, .2],
                        size=[.1, .1, 0])

model = spec.compile()
render(model)

`mjSpec` can save XML to string, saving all modifications.

In [ ]:
#@title Print an XML from an `mjSpec` {vertical-output: true}

print_xml(spec.to_xml())

In [ ]:
#@title Building an `mjSpec` from scratch {vertical-output: true}

spec = mj.MjSpec()
spec.worldbody.add_light(name="top", pos=[0, 0, 1])
body = spec.worldbody.add_body(name="box_and_sphere", euler=[0, 0, -30])
body.add_joint(name="swing", type=mj.mjtJoint.mjJNT_HINGE,
               axis=[1, -1, 0], pos=[-.2, -.2, -.2])
body.add_geom(name="red_box", type=mj.mjtGeom.mjGEOM_BOX,
              size=[.2, .2, .2], rgba=[1, 0, 0, 1])
body.add_geom(name="green_sphere", pos=[.2, .2, .2],
              size=[.1, 0, 0], rgba=[0, 1, 0, 1])
model = spec.compile()

duration = 2    # (seconds)
framerate = 30  # (Hz)

# enable joint visualization option:
scene_option = mj.MjvOption()
scene_option.flags[mj.mjtVisFlag.mjVIS_JOINT] = True

# Simulate and display video.
frames = []
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)


# Control example

A key feature of this library is the ability to easily attach multiple models into a larger one. Disambiguation of duplicated names from different
models, or multiple instances of the same model, is handled via user-defined namespacing.

One example use case is when we want robots with a variable number of joints, as is a fundamental change to the kinematic structure. The following snippets realise this scenario.

In [ ]:
leg_model = """
<mujoco>
  <compiler angle="radian"/>

  <default>
    <joint damping="2" type="hinge"/>
    <geom type="capsule"/>
  </default>

  <worldbody>
    <body name="thigh">
      <joint name="hip" axis="0 0 1"/>
      <body name="shin">
        <joint name="knee" axis="0 1 0"/>
      </body>
    </body>
  </worldbody>

  <actuator>
    <position joint="hip" kp="10"/>
    <position joint="knee" kp="10"/>
  </actuator>
</mujoco>
"""

class Leg(object):
  """A 2-DoF leg with position actuators."""
  def __init__(self, length, rgba):
    self.spec = mj.MjSpec.from_string(leg_model)

    # Thigh:
    thigh = self.spec.find_body('thigh')
    thigh.add_geom(fromto=[0, 0, 0, length, 0, 0], size=[length/4, 0, 0], rgba=rgba)

    # Hip:
    shin = self.spec.find_body('shin')
    shin.add_geom(fromto=[0, 0, 0, 0, 0, -length], size=[length/5, 0, 0], rgba=rgba)
    shin.pos[0] = length

The `Leg` class describes an abstract articulated leg, with two joints and corresponding proportional-derivative actuators.

Note that:

- MJCF attributes correspond directly to arguments of the `add_()` methods.
- When referencing elements, e.g when specifying the joint to which an actuator is attached, the name string of the MJCF elements is used.

In [ ]:
BODY_RADIUS = 0.1
random_state = np.random.RandomState(42)
creature_model = """
<mujoco>
  <compiler angle="radian"/>

  <worldbody>
    <geom name="torso" type="ellipsoid" size="{} {} {}"/>
  </worldbody>
</mujoco>
""".format(BODY_RADIUS, BODY_RADIUS, BODY_RADIUS / 2)

def make_creature(num_legs):
  """Constructs a creature with `num_legs` legs."""
  rgba = random_state.uniform([0, 0, 0, 1], [1, 1, 1, 1])
  spec = mj.MjSpec.from_string(creature_model)

  # Attach legs to equidistant sites on the circumference.
  spec.worldbody.first_geom().rgba = rgba
  leg = Leg(length=BODY_RADIUS, rgba=rgba)
  for i in range(num_legs):
    theta = 2 * i * np.pi / num_legs
    hip_pos = BODY_RADIUS * np.array([np.cos(theta), np.sin(theta), 0])
    hip_site = spec.worldbody.add_site(pos=hip_pos, euler=[0, 0, theta])
    hip_site.attach(leg.spec.find_body('thigh'), '', '-' + str(i))

  return spec

The `make_creature` function uses the `attach()` method to procedurally attach legs to the torso. Note that at this stage both the torso and hip attachment sites are children of the `worldbody`, since their parent body has yet to be instantiated. We'll now make an arena with a chequered floor and two lights, and place our creatures in a grid.

In [ ]:
#@title Six Creatures on a floor.{vertical-output: true}

arena = mj.MjSpec()

if hasattr(arena, 'compiler'):
  arena.compiler.degree = False  # MuJoCo dev (next release).
else:
  arena.degree = False  # MuJoCo release

# Make arena with textured floor.
chequered = arena.add_texture(
    name="chequered", type=mj.mjtTexture.mjTEXTURE_2D,
    builtin=mj.mjtBuiltin.mjBUILTIN_CHECKER,
    width=300, height=300, rgb1=[.2, .3, .4], rgb2=[.3, .4, .5])
grid = arena.add_material(
    name='grid', texrepeat=[5, 5], reflectance=.2
    ).textures[mj.mjtTextureRole.mjTEXROLE_RGB] = 'chequered'
arena.worldbody.add_geom(
    type=mj.mjtGeom.mjGEOM_PLANE, size=[2, 2, .1], material='grid')
for x in [-2, 2]:
  arena.worldbody.add_light(pos=[x, -1, 3], dir=[-x, 1, -2])

# Instantiate 6 creatures with 3 to 8 legs.
creatures = [make_creature(num_legs=num_legs) for num_legs in range(3, 9)]

# Place them on a grid in the arena.
height = .15
grid = 5 * BODY_RADIUS
xpos, ypos, zpos = np.meshgrid([-grid, 0, grid], [0, grid], [height])
for i, spec in enumerate(creatures):
  # Place spawn sites on a grid.
  spawn_pos = (xpos.flat[i], ypos.flat[i], zpos.flat[i])
  spawn_site = arena.worldbody.add_site(pos=spawn_pos, group=3)
  # Attach to the arena at the spawn sites, with a free joint.
  spawn_body = spawn_site.attach(spec.worldbody, '', '-' + str(i))
  spawn_body.add_freejoint()

# Instantiate the physics and render.
model = arena.compile()
render(model)

Multi-legged creatures, ready to roam! Let's inject some controls and watch them move. We'll generate a sinusoidal open-loop control signal of fixed frequency and random phase, recording both video frames and the horizontal positions of the torso geoms, in order to plot the movement trajectories.

In [ ]:
#@title Video of the movement{vertical-output: true}

duration = 10   # (Seconds)
framerate = 30  # (Hz)
video = []
pos_x = []
pos_y = []
geoms = arena.worldbody.find_all(mj.mjtObj.mjOBJ_GEOM)
torsos = [geom.id for geom in geoms if 'torso' in geom.name]
actuators = [actuator.id for actuator in arena.actuators]

# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.actuators))
amp = 0.9

# Simulate, saving video frames and torso locations.
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    data.ctrl[actuators] = amp * np.sin(freq * data.time + phase)
    mj.mj_step(model, data)

    # Save torso horizontal positions using name indexing.
    pos_x.append(data.geom_xpos[torsos, 0].copy())
    pos_y.append(data.geom_xpos[torsos, 1].copy())

    # Save video frames.
    if len(video) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      video.append(pixels.copy())

media.show_video(video, fps=framerate)

In [ ]:
#@title Movement trajectories{vertical-output: true}

creature_colors = model.geom_rgba[torsos][:, :3]
fig, ax = plt.subplots(figsize=(4, 4))
ax.set_prop_cycle(color=creature_colors)
_ = ax.plot(pos_x, pos_y, linewidth=4)

The plot above shows the corresponding movement trajectories of creature positions. Note how `mjSpec` attribute `id` were used to access both `xpos` and `rgba` values. This attribute is valid only after a model is compiled.

# Model editing

First we need to download models from mujoco_menagerie

In [ ]:
# Get MuJoCo's humanoid model and a Franka arm from the MuJoCo Menagerie.
print('Getting MuJoCo humanoid XML description from GitHub:')
!git clone https://github.com/google-deepmind/mujoco
humanoid_file = 'mujoco/model/humanoid/humanoid.xml'
humanoid100_file = 'mujoco/model/humanoid/humanoid100.xml'
print('Getting MuJoCo Menagerie Franka XML description from GitHub:')
!git clone https://github.com/google-deepmind/mujoco_menagerie
franka_file = 'mujoco_menagerie/franka_fr3/fr3.xml'

`mjSpec` elements can be traversed in two ways:
- For elements inside the kinematic tree, the tree can be traversed using the `first` and `next` functions.
- For all other elements, we provide a list.



In [ ]:
#@title Traversing the spec.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)

# Function that recursively prints all body names
def print_bodies(parent, level=0):
  body = parent.first_body()
  while body:
    print(''.join(['-' for i in range(level)]) + body.name)
    print_bodies(body, level + 1)
    body = parent.next_body(body)

print("The spec has the following actuators:")
for actuator in spec.actuators:
  print(actuator.name)

print("\nThe spec has the following bodies:")
print_bodies(spec.worldbody)

An `mjSpec` can be compiled multiple times. If the state has to be preserved between different compilations, then the function `recompile()` must be used, which returns a new `mjData` that contains the mapped state, possibly having a different dimension from the origin.

In [ ]:
#@title Model re-compilation with state preservation.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid100_file)
model = spec.compile()
data = mj.MjData(model)

# Run for 5 seconds
for i in range(1000):
  mj.mj_step(model, data)

# Show result
render(model, data)

# Create list of all bodies we want to delete
body = spec.worldbody.first_body()
delete_list = []
while body:
  geom_type = body.first_geom().type
  if (geom_type == mj.mjtGeom.mjGEOM_BOX or
      geom_type == mj.mjtGeom.mjGEOM_ELLIPSOID):
    delete_list.append(body)
  body = spec.worldbody.next_body(body)

# Remove all bodies in the list from the spec
for body in delete_list:
  spec.detach_body(body)

# # Add another humanoid
spec_humanoid = mj.MjSpec.from_file(humanoid_file)
attachment_frame = spec.worldbody.add_frame(pos=[0, -1, 2])
attachment_frame.attach_body(spec_humanoid.find_body('torso'), 'a', 'b')

# Recompile preserving the state
new_model, new_data = spec.recompile(model, data)

# Show result
render(new_model, new_data)

Let us load the humanoid model and inspect it.

In [ ]:
#@title Humanoid model.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)

model = spec.compile()
render(model)

We wish to remove the arms and replace them with the legs. This can be done by first storing the arm positions into frames attaches to the torso. Then we can detach the arms and self-attach the legs into the frames.

In [ ]:
#@title Humanoid with arms replaced by legs.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)

# Get the torso, arm, and leg bodies
arm_left = spec.find_body('upper_arm_left')
arm_right = spec.find_body('upper_arm_right')
leg_left = spec.find_body('thigh_left')
leg_right = spec.find_body('thigh_right')
torso = spec.find_body('torso')

# Attach frames at the arm positions
shoulder_left = torso.add_frame(pos=arm_left.pos)
shoulder_right = torso.add_frame(pos=arm_right.pos)

# Remove the arms
spec.detach_body(arm_left)
spec.detach_body(arm_right)

# Add new legs
shoulder_left.attach_body(leg_left, 'shoulder', 'left')
shoulder_right.attach_body(leg_right, 'shoulder', 'right')

model = spec.compile()
render(model, height=400)

Similarly, different models can be attach together. Here, the right arm is detached and a robot arm from a different model is attached in its place.

In [ ]:
#@title Humanoid with Franka arm.{vertical-output: true}

spec = mj.MjSpec.from_file(humanoid_file)
franka = mj.MjSpec.from_file(franka_file)

if hasattr(spec, 'compiler'):
  spec.compiler.degree = False  # MuJoCo dev (next release).
else:
  spec.degree = False  # MuJoCo release

# Replace right arm with frame
arm_right = spec.find_body('upper_arm_right')
torso = spec.find_body('torso')
shoulder_right = torso.add_frame(pos=arm_right.pos, quat=[0, 0.8509035, 0, 0.525322])
spec.detach_body(arm_right)

# Attach Franka arm to humanoid
franka_arm = franka.find_body('fr3_link2')
shoulder_right.attach_body(franka_arm, 'franka', '')

model = spec.compile()
render(model, height=400)

When doing this, the actuators and all other objects referenced by the attached sub-tree are imported in the new model. All assets are currently imported, referenced or not.

In [ ]:
#@title Imported actuators.{vertical-output: true}

for actuator in spec.actuators:
  print(actuator.name)

Domain randomization can be performed by attaching multiple times the same spec, edited each time with a new instance of randomized parameters.

In [ ]:
#@title Humanoid with randomized heads and arm poses.{vertical-output: true}

humanoid = mj.MjSpec.from_file(humanoid_file)
spec = mj.MjSpec()

# Delete all key frames to avoid name conflicts
while humanoid.keys:
  humanoid.keys[-1].delete()

# Create a grid of humanoids by attaching humanoid to spec multiple times
for i in range(4):
  for j in range(4):
    humanoid.materials[0].rgba = [
        np.random.uniform(), np.random.uniform(),
        np.random.uniform(), 1]  # Randomize color
    humanoid.find_body('head').first_geom().size = [
        .18*np.random.uniform(), 0, 0]  # Randomize head size
    humanoid.find_body('upper_arm_left').quat = [
        np.random.uniform(), np.random.uniform(),
        np.random.uniform(), np.random.uniform()]  # Randomize left arm orientation
    humanoid.find_body('upper_arm_right').quat = [
        np.random.uniform(), np.random.uniform(),
        np.random.uniform(), np.random.uniform()]  # Randomize right arm orientation

    # attach randomized humanoid to parent spec
    frame = spec.worldbody.add_frame(pos=[i, j, 0])
    frame.attach_body(humanoid.find_body('torso'), str(i), str(j))

spec.worldbody.add_light(mode=mj.mjtCamLight.mjCAMLIGHT_TARGETBODYCOM,
                         targetbody='3torso3', diffuse=[.8, .8, .8],
                         specular=[0.3, 0.3, 0.3], pos=[0, -6, 4], cutoff=30)
model = spec.compile()
render(model, height=400)

# Pure Models with `mjspec`

A robot operates within an enviornment. We refer to this enviornment as Arena. To add versatility to the enviornment we should be able to add or remove things from enviorment. To add these functionalities to the Arena we will use mjspec's attach, detach and frame functionality.

## Arena

In [ ]:
class Arena:
  def __init__(self):
    self.spec = mj.MjSpec()
    self.model = self.spec.worldbody

    self.spec.compiler.degree = False

    # Make arena with textured floor.
    chequered = self.spec.add_texture(
        name="chequered", type=mj.mjtTexture.mjTEXTURE_2D,
        builtin=mj.mjtBuiltin.mjBUILTIN_CHECKER,
        width=300, height=300, rgb1=[.2, .3, .4], rgb2=[.3, .4, .5])
    grid = self.spec.add_material(
        name='grid', texrepeat=[5, 5], reflectance=.2
        ).textures[mj.mjtTextureRole.mjTEXROLE_RGB] = 'chequered'
    self.model.add_geom(
        type=mj.mjtGeom.mjGEOM_PLANE, size=[2, 2, .1], material='grid')
    for x in [-2, 2]:
      self.model.add_light(pos=[x, -1, 3], dir=[-x, 1, -2])

  def add_movable_asset(self,asset,pos=[0,0,0],prefix='_'):
    frame = self.model.add_frame(pos=pos)
    body = frame.attach_body(asset.model,  prefix,'')
    body.add_freejoint()
    return body

  def add_fixed_asset(self,asset,pos=[0,0,0],prefix='_'):
    frame = self.model.add_frame(pos=pos)
    body = frame.attach_body(asset.model,  prefix,'')
    return body

  def remove_asset(self,body):
    self.spec.detach_body(body)

In [ ]:
arena = Arena()
model = arena.spec.compile()
render(model)

## Creating a Robot
Next, we will look at how to put a robot together. We are going to create a quadruped, which has four identical legs. We break down the leg into three parts, namely the robot tree structure, actuation, and sensors. We will implement tendon-driven and position control actuation for the leg.

### Robot tree structure
we start by setting default values such as color and joint damping, which are shared throughout the entire model. Next, we proceed to define the data belonging to the tree, referred to as properties. Separating the data from the tree-like structure enables quick modifications to the model if needed. Finally, the properties are used to construct the model.

In [ ]:
class LegBase:
  def __init__(self,length=0.1):
    self.spec = mj.MjSpec()
    self.spec.compiler.degree = False
    self.model = self.spec.worldbody

    # material
    self.spec.add_material(name = "yellow",rgba=[1,1,0,1])

    # defaults
    main = self.spec.default()
    main.joint.damping = 2

    # props
    bodies = [
        {
            "body":{
                "name":"thigh"
            },
            "joint":{
                "name":"hip",
                "axis":[0,0,1],
            },
            "geom":{
                "fromto":[0]*3+[length]+[0]*2,
                "size":[length/4,0,0],
                "material":"yellow",
                "type":mj.mjtGeom.mjGEOM_CAPSULE,
                "mass":1
            }
        },
        {
            "body":{
                "name":"shin",
                "pos":[length,0,0]
            },
            "joint":{
                "name":"knee",
                "axis":[0, 1, 0],
            },
            "geom":{
                "fromto":[0]*5+[-length],
                "size":[length/5,0,0],
                "material":"yellow",
                "type":mj.mjtGeom.mjGEOM_CAPSULE,
                "mass":1

            },
            "site":{
                "name":"feet",
                "pos":[0,0,-length]
            }
        }
    ]
    body = self.model
    for prop in bodies:
      body = body.add_body(**prop['body'])
      body.add_joint(**prop['joint'])
      body.add_geom(**prop['geom'])
      if  "site" in prop:
        body.add_site(**prop['site'])

In [ ]:
leg = LegBase()
model = leg.spec.compile()
print_xml(leg.spec.to_xml())

Next lets have a look at how we can add and remove a model from arena

In [ ]:
arena = Arena()
body = arena.add_fixed_asset(leg,pos=[0,0,0.15])
model = arena.spec.compile()
render(model)

In [ ]:
arena.remove_asset(body)
model = arena.spec.compile()
render(model)

### Actuator
Here we add positon actuaotors to the model. We start by drving the LegActuated class from LegBase. We need create a position actuator from a general actuator. We achived this by setting default values of the general actuator.

In [ ]:
class LegActuation(LegBase):
  def __init__(self,length=0.1,kp=10):
    super().__init__(length)

    # defaults
    main = self.spec.default()

    # position actuator defaults
    main.actuator.trntype = mj.mjtTrn.mjTRN_JOINT
    main.actuator.gaintype = mj.mjtGain.mjGAIN_FIXED
    main.actuator.biastype = mj.mjtBias.mjBIAS_AFFINE
    main.actuator.gainprm[0] =  kp
    main.actuator.biasprm[1] = -kp

    actuators =[
        {
            "name":"hip_act",
            "target":"hip"
        },
        {
            "name":"knee_act",
            "target":"knee"
        },
    ]

    for prop in actuators:
      self.spec.add_actuator(**prop)

In [ ]:
leg = LegActuation()
model = leg.spec.compile()
print_xml(leg.spec.to_xml())

### Sensors
To better understand what is going on with the robot we need to add some sensors. Mujoco support different types of sensors that can be found in: mujoco/introspect
/enums.py

In order to use a sensor we need to define sensor type, object that it operates on and object name. For example a joint position sensor will have a type `mjtSensor.mjSENS_JOINTPOS` operate on object of type `mjtObj.mjOBJ_JOINT` and a name `super_duper_unique_joint_name`


In [ ]:
class Leg(LegActuation):
  def __init__(self,length=0.1,kp=10,rgba=[1,0,0,1]):
    super().__init__(length,kp)

    # props
    sesnros = [
      # joint sensors
      {
          "name":"hip_jpos",
          "objname":"hip",
          "type":mj.mjtSensor.mjSENS_JOINTPOS,
          "objtype":mj.mjtObj.mjOBJ_JOINT

      },
      {
          "name":"hip_jvel",
          "objname":"hip",
          "type":mj.mjtSensor.mjSENS_JOINTVEL,
          "objtype":mj.mjtObj.mjOBJ_JOINT

      },
      # actuator sensors
       {
          "name":"hip_apos",
          "objname":"hip_act",
          "type":mj.mjtSensor.mjSENS_ACTUATORPOS,
          "objtype":mj.mjtObj.mjOBJ_ACTUATOR

      },
      {
          "name":"hip_avel",
          "objname":"hip_act",
          "type":mj.mjtSensor.mjSENS_ACTUATORVEL,
          "objtype":mj.mjtObj.mjOBJ_ACTUATOR

      },
      # touch sensor
      {
          "name":"feet_touch",
          "objname":"feet",
          "type":mj.mjtSensor.mjSENS_TOUCH,
          "objtype":mj.mjtObj.mjOBJ_SITE

      },
      # force sensor
      {
          "name":"feet_force",
          "objname":"feet",
          "type":mj.mjtSensor.mjSENS_FORCE,
          "objtype":mj.mjtObj.mjOBJ_SITE

      },
    ]

    for prop in sesnros:
      self.spec.add_sensor(**prop)

In [ ]:
leg = Leg()
model = leg.spec.compile()
print_xml(leg.spec.to_xml())

In [ ]:
arena = Arena()
robot = Leg()
arena.add_fixed_asset(robot,[0,0,0.11])
model = arena.spec.compile()
random_state = np.random.RandomState(42)

actuators = [actuator.id for actuator in arena.spec.actuators]
sensors = dict(zip(
    [sensor.name for sensor in arena.spec.sensors],
    [sensor.id for sensor in arena.spec.sensors]
))

print(f"Sensors: {sensors}")

# Control signal frequency, phase, amplitude.
freq = 5
phase = 2 * np.pi * random_state.rand(len(arena.spec.actuators))
amp = 0.9

# Simulate and display video.
duration = 3    # (seconds)
framerate = 30  # (Hz)
frames = []
hip_jpos = []
data = mj.MjData(model)
mj.mj_resetData(model, data)
with mj.Renderer(model) as renderer:
  while data.time < duration:
    # Inject controls and step the physics.
    data.ctrl[actuators] = amp * np.sin(freq * data.time + phase)
    mj.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data)
      pixels = renderer.render()
      frames.append(pixels)
      hip_jpos.append(data.sensordata[sensors['_hip_apos']])

media.show_video(frames, fps=framerate)

fig, ax = plt.subplots(figsize=(4, 2))
_ = ax.plot( range(len(hip_jpos)),hip_jpos, linewidth=1)
ax.set_ylabel('hip joint position')

